Task 2

In [23]:
from pyspark.sql import SparkSession
from functools import reduce

def load_dataset(filename, columnNames):
    df = sc.textFile(filename)
    return df

spark = SparkSession.builder.appName("MapReduce").getOrCreate()
sc = spark.sparkContext

columns = ['passengerID', 'flightID', 'originAirport', 'destinationAirport', 'departureTime', 'flightTime']

rdd = load_dataset("AComp_Passenger_data_no_error.csv", columns)
rdd.foreach(lambda x: print(x))



JBE2302VO4,KJR6646J,IAH,BKK,1420565203,1928
CXN7304ER2,VYW5940P,LAS,SIN,1420565203,1843
VZY2993ME1,XIL3623J,PEK,LAX,1420564414,1302
EDV2089LK5,YZO4444S,BKK,MIA,1420565330,2027
DAZ3029XA0,KJR6646J,IAH,BKK,1420565203,1928UES9151GS5,SQU6245R,DEN,FRA,1420564460,1049
UES9151GS5,XXQ4064B,JFK,FRA,1420563917,802
EZC9678QI6,SOH3431A,ORD,MIA,1420563649,250
ONL0812DH1,SOH3431A,ORD,MIA,1420563649,250
CYJ0225CH1,PME8178S,DEN,PEK,1420564409,1322
POP2875LH3,MBA8071P,KUL,PEK,1420563856,572
WTC9125IE5,MOO1786A,MAD,FRA,1420563408,184
EDV2089LK5,HUR0974O,DEN,PVG,1420564525,1398
UES9151GS5,GMO5938W,LHR,PEK,1420564317,1057
HCA3158QA6,XXQ4064B,JFK,FRA,1420563917,802
YMH6360YP0,DAU2617A,CGK,SFO,1420564986,1811
PUD8209OG3,RUM0422W,MUC,MAD,1420563539,194
PAJ3974RK1,ATT7791R,AMS,DEN,1420564394,1001
UES9151GS5,WPW9201U,DFW,PEK,1420564869,1452
WYU2010YH8,XXQ4064B,JFK,FRA,1420563917,802
JJM4724RF7,HUR0974O,DEN,PVG,1420564525,1398
MXU9187YC7,XXQ4064B,JFK,FRA,1420563917,802
HGO4350KK1,RUM0422W,MUC,MAD,1420563539,194